In [1]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace


df_soil = pd.read_csv('LUCAS-SOIL-2018.csv')
df_crops = pd.read_csv('AllCrops2020.csv')

# Namespaces
SLO = Namespace("https://w3id.org/soilkg/ontology/")
SCHEMA = Namespace("http://schema.org/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")

# Initialize the graph
ont = Graph()
ont.parse('soil_kg_ontology.ttl', format='turtle')


<Graph identifier=N45e9034fb7fb4a129e402721238bd8e5 (<class 'rdflib.graph.Graph'>)>

### CONSTRUCTING ONTOLOGY

The ontology was constructed by adding all classes like soil properties, crops, land cover and land use. Some classes were added by hand and some with a python script. 

In [2]:
df_crops.head()

,REGION,CROP_NAME,YEAR,VARIABLE,VALUE,UoM,SOURCE,CALCULATED_R,CALCULATED_C,CALCULATED_V,ZERO_AS_NULL,COHERENCE_APY,COHERENCE_CROP
0,AT11,Total wheat,2020,Area,40620.67,ha,NSI,NaN,NaN,Yes,NaN,Yes,Yes
1,AT11,Soft wheat,2020,Area,37340.26,ha,NSI,NaN,Yes,NaN,NaN,Yes,Yes
2,AT11,Durum wheat,2020,Area,3280.41,ha,NSI,NaN,Yes,NaN,NaN,Yes,Yes
3,AT11,Total wheat,2020,Production,212587.66,t,NSI,NaN,NaN,Yes,NaN,Yes,Yes
4,AT11,Soft wheat,2020,Production,198902.60,t,NSI,NaN,Yes,NaN,NaN,Yes,Yes


In [3]:
# return rows that have missing values
df_soil.head()

,Depth,POINTID,pH_CaCl2,pH_H2O,EC,OC,CaCO3,P,N,K,...,NUTS_3,TH_LAT,TH_LONG,SURVEY_DATE,Elev,LC,LU,LC0_Desc,LC1_Desc,LU1_Desc
0,0-20 cm,47862690,4.1,4.81,8.73,12.4,3,< LOD,1.1,101.9,...,AT113,47.150238,16.134212,06-07-18,291,C23,U120,Woodland,Other coniferous woodland,Forestry
1,0-20 cm,47882704,4.1,4.93,5.06,16.7,1,< LOD,1.3,51.2,...,AT113,47.274272,16.175359,06-07-18,373,C21,U120,Woodland,Spruce dominated coniferous woodland,Forestry
2,0-20 cm,47982688,4.1,4.85,12.53,47.5,1,12.3,3.1,114.8,...,AT113,47.123260,16.289693,02-06-18,246,C33,U120,Woodland,Other mixed woodland,Forestry
3,0-20 cm,48022702,5.5,5.80,21.10,28.1,3,< LOD,2,165.8,...,AT113,47.245693,16.357506,06-07-18,305,C22,U120,Woodland,Pine dominated coniferous woodland,Forestry
4,0-20 cm,48062708,6.1,6.48,10.89,19.4,2,< LOD,2.2,42.1,...,AT113,47.296372,16.416782,05-07-18,335,C22,U120,Woodland,Pine dominated coniferous woodland,Forestry


### CONSTRUCTING KNOWLEDGE GRAPH

Based on the ontology the Knowledge Graph is  constructed: 

In [5]:
from urllib.parse import quote
from datetime import datetime
kg = Graph()

# Add namespaces to the graph
kg.bind('slo', SLO)
kg.bind('schema', SCHEMA)
kg.bind('rdfs', RDFS)
kg.bind('xsd', XSD)


def create_uri(class_name, value):
    return URIRef(f"https://w3id.org/soilkg/{class_name}/{value}")

# soil properties
soil_properties = { 'OC': SLO.OC, 'EC': SLO.EC, 'pH_H2O': SLO.PHH2O, 'pH_CaCl2': SLO.PHCaCl2, 'CaCO3': SLO.CaCO3, 
                       'P': SLO.P, 'K': SLO.K, 'N': SLO.N, 'OC (20-30 cm)': SLO.OCC20_30, 'CaCO3 (20-30 cm)': SLO.CaCO3C20_30, 
                    'Ox_Al': SLO.Ox_Al, 'Ox_Fe': SLO.Ox_Fe,
            }
    
# ADD SOIL DATA
for index, row in df_soil.iterrows():
    
    # Create URI for the soil measurement
    soil_measurement_uri = create_uri('SoilSurvey', row['POINTID'])
    kg.add((soil_measurement_uri, RDF.type, SLO.SurveyPoint))

    # Add NUT regions
    nuts_0_region_uri = create_uri('NUTS0', row['NUTS_0'])   
    nuts_1_region_uri = create_uri('NUTS1', row['NUTS_1'])
    nuts2_region_uri = create_uri('NUTS2', row['NUTS_2'])
    nuts3_region_uri = create_uri('NUTS3', row['NUTS_3'])
    kg.add((nuts_0_region_uri, RDF.type, SLO.NUTS0))
    kg.add((nuts_1_region_uri, RDF.type, SLO.NUTS1))
    kg.add((nuts2_region_uri, RDF.type, SLO.NUTS2))
    kg.add((nuts3_region_uri, RDF.type, SLO.NUTS3))

    # Add location
    kg.add((soil_measurement_uri, SLO.surveyLocation, nuts2_region_uri))
    kg.add((soil_measurement_uri, SLO.surveyLocation, nuts3_region_uri))

    # Get soil properties for this row and add them to the graph
    for property_name, property_uri in soil_properties.items():
        soil_property_value = row[property_name]
        # if not integer or float, skip
        if not isinstance(soil_property_value, (int, float)):
            continue
        # if nan, skip
        if pd.isna(soil_property_value):
            continue
        soil_property_uri = create_uri('SoilProperty', f'{quote(property_name)}/{row["POINTID"]}')
        kg.add((soil_property_uri, RDF.type, property_uri))
        kg.add((soil_property_uri, SCHEMA.value, Literal(soil_property_value, datatype=XSD.float)))
        kg.add((soil_measurement_uri, SLO.hasProperty, soil_property_uri))
    
    # Add date 
    date = row['SURVEY_DATE']
    date = datetime.strptime(date, '%d-%m-%y').strftime('%Y-%m-%d')
    if not pd.isna(date):
        kg.add((soil_measurement_uri, SLO.surveyDate, Literal(date, datatype=XSD.date)))
    
    # Add land use and land cover
    land_use = row['LC']
    land_cover = row['LU']
    if not pd.isna(land_use):
        land_use_uri =  URIRef(f'https://w3id.org/soilkg/ontology/LU_{land_use}')
    if not pd.isna(land_cover):
        land_cover_uri =  URIRef(f'https://w3id.org/soilkg/ontology/LC_{land_cover}')
    kg.add((soil_measurement_uri, SLO.hasLU, land_use_uri))
    kg.add((soil_measurement_uri, SLO.hasLC, land_cover_uri))

    # Add elevation
    elevation = row['Elev']
    if not pd.isna(elevation):
        kg.add((soil_measurement_uri, SLO.hasElevation, Literal(elevation, datatype=XSD.float)))

# ADD CROP DATA
for index, row in df_crops.iterrows():
    #get all information from the row
    NUT2 = row['REGION']
    variable = row['VARIABLE']
    crop = row['CROP_NAME'].replace(" ", "_")
    date = row['YEAR']
    value = row['VALUE']

    # Create URI for the crop measurement
    crop_measurement_uri = create_uri('CropMeasurement', f'{crop}/{NUT2}/{variable}')

    # Create URI for the crop
    crop_name_class = URIRef(f'https://w3id.org/soilkg/ontology/{crop}')

    # Create URI for the NUT2 region
    NUT2_uri = create_uri('NUTS2', NUT2)

    # Creat URI for the variable
    crop_variable_class = URIRef(f'https://w3id.org/soilkg/ontology/{variable}')

    # Add the crop measurement to the graph                                    
    kg.add((crop_measurement_uri, RDF.type, SLO.CropMeasurement))
    if not pd.isna(NUT2):
        kg.add((crop_measurement_uri, SLO.CropMeasurementLocation, NUT2_uri))
    if not pd.isna(date):
        kg.add((crop_measurement_uri, SLO.CropMeasurementDate, Literal(date, datatype=XSD.date)))
    if not pd.isna(value):
        kg.add((crop_measurement_uri, SCHEMA.value, Literal(value, datatype=XSD.float)))
    if not pd.isna(crop):
        kg.add((crop_measurement_uri, SLO.cropMeasured, crop_name_class))
    if not pd.isna(variable):
        kg.add((crop_measurement_uri, SLO.hasVariable, crop_variable_class))
    
    
    

# Serialize the graph in Turtle format and save it to a file
kg.serialize(destination='soil_kg.ttl', format='turtle')


<Graph identifier=N81f4ee4a9e9b4c8cb88e380e8d25bdf3 (<class 'rdflib.graph.Graph'>)>

In [6]:
kg.parse('soil_kg_ontology.ttl', format='turtle')

<Graph identifier=N81f4ee4a9e9b4c8cb88e380e8d25bdf3 (<class 'rdflib.graph.Graph'>)>

### DATA QUALITY ASSESEMENT

Here I assess the intrinsic qualities of the graph namely: 
- Syntactic validity 
- Consistency
- Conciseness
- Semantic Accuracy 
- Completeness

#### SYNTACTIC VALIDITY

In [7]:
# Syntactic Validity
# Validate that all the literal values are of the correct datatype
# Validate that all the URIs are valid
from pyshacl import validate
shapes_file = 'validity_shapes.ttl'

r = validate('soil_kg.ttl', shacl_graph=shapes_file, ont_graph=ont, inference='rdfs', abort_on_first=False, allow_warnings=True)
print(r[2])

Validation Report
Conforms: True



#### SEMANTIC ACCURACY

In [8]:
# Semantic Accuracy
# Validate that the values of the literals are within reasonable ranges, e.g., pH values between 0 and 14. 
from pyshacl import validate
shapes_file = 'accuracy_shapes.ttl'

r = validate('soil_kg.ttl', shacl_graph=shapes_file, ont_graph=ont, inference='rdfs', abort_on_first=False, allow_warnings=True)
print(r[2])


Validation Report
Conforms: False
Results (402):
Constraint Violation in MaxInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MaxInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:float ; sh:maxInclusive Literal("4809", datatype=xsd:integer) ; sh:minInclusive Literal("-293", datatype=xsd:integer) ; sh:name Literal("Elevation") ; sh:path slo:hasElevation ]
	Focus Node: <https://w3id.org/soilkg/SoilSurvey/51424726>
	Value Node: Literal("10187.0", datatype=xsd:float)
	Result Path: slo:hasElevation
	Message: Value is not <= Literal("4809", datatype=xsd:integer)
Constraint Violation in MaxInclusiveConstraintComponent (http://www.w3.org/ns/shacl#MaxInclusiveConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:float ; sh:maxInclusive Literal("4809", datatype=xsd:integer) ; sh:minInclusive Literal("-293", datatype=xsd:integer) ; sh:name Literal("Elevation") ; sh:path slo:hasElevation ]
	Focus Node: <https://w3id.org/soilkg/S

#### CONCISENESS

In [9]:
# Conciseness
prefixes = """ 
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX ramon: <http://rdfdata.eionet.europa.eu/ramon/ontology/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX slo: <https://w3id.org/soilkg/ontology/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
"""
# Check if there are duplicate triples
query =  prefixes + """
SELECT ?s ?p ?o (COUNT(*) AS ?count)
WHERE {
  ?s ?p ?o .
}
GROUP BY ?s ?p ?o
HAVING (COUNT(*) > 1)
"""

results = kg.query(query)
for row in results:
    print(row)

In [10]:
# Check instances with high number of properties
query = prefixes + """
SELECT ?s (COUNT(DISTINCT ?p) AS ?propertyCount)
WHERE {
  ?s ?p ?o .
}
GROUP BY ?s
ORDER BY DESC(?propertyCount)
LIMIT 10
"""
results = kg.query(query)
for row in results:
    print(row)

(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/33641952'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/31801876'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/43022764'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/52423856'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/47303242'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/soilkg/SoilSurvey/45002146'), rdflib.term.Literal('7', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))

In [11]:
# Check if there are instances with multiple types
query = prefixes + """
SELECT ?s (GROUP_CONCAT(DISTINCT ?type; separator=", ") AS ?types)
WHERE {
  ?s a ?type .
}
GROUP BY ?s
HAVING (COUNT(?type) > 1)
"""
results = kg.query(query)
for row in results:
    print(row)

(rdflib.term.URIRef('https://w3id.org/soilkg/ontology/NUTS0'), rdflib.term.Literal('http://www.w3.org/2002/07/owl#Class, http://rdfdata.eionet.europa.eu/ramon/ontology/NUTSRegion'))
(rdflib.term.URIRef('https://w3id.org/soilkg/ontology/NUTS1'), rdflib.term.Literal('http://www.w3.org/2002/07/owl#Class, http://rdfdata.eionet.europa.eu/ramon/ontology/NUTSRegion'))
(rdflib.term.URIRef('https://w3id.org/soilkg/ontology/NUTS2'), rdflib.term.Literal('http://www.w3.org/2002/07/owl#Class, http://rdfdata.eionet.europa.eu/ramon/ontology/NUTSRegion'))
(rdflib.term.URIRef('https://w3id.org/soilkg/ontology/NUTS3'), rdflib.term.Literal('http://www.w3.org/2002/07/owl#Class, http://rdfdata.eionet.europa.eu/ramon/ontology/NUTSRegion'))


In [12]:
# Check if an object has same value described by two different properties
query = prefixes + """
SELECT ?s ?o1 ?o2
WHERE {
  ?s ?p1 ?o1 .
  ?s ?p2 ?o2 .
  FILTER(?p1 != ?p2 && STR(?o1) = STR(?o2))
}
LIMIT 100
"""
results = kg.query(query)
for row in results:
    print(row)

#### CONSISTENCY

In [14]:
# Consistency
# Validate that the data is consistent with the ontology, e.g., that all the properties of a class are correctly used.
shapes_file = 'consistency_shapes.ttl'

r = validate('soil_kg.ttl', shacl_graph=shapes_file, ont_graph=ont, inference='rdfs', abort_on_first=False, allow_warnings=True)
print(r[2])

Validation Report
Conforms: False
Results (16):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class slo:SoilProperty ; sh:maxCount Literal("16", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:name Literal("Soil Property") ; sh:path slo:hasProperty ]
	Focus Node: <https://w3id.org/soilkg/SoilSurvey/27021938>
	Result Path: slo:hasProperty
	Message: Less than 1 values on <https://w3id.org/soilkg/SoilSurvey/27021938>->slo:hasProperty
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:float ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:name Literal("Value") ; sh:path schema1:value ]
	Focus Node: <https://w3id.org/soilkg/CropMeasurement/Sunflower/NL42/Production>
	Result Path: schema1:va

#### COMPLETENESS

In [16]:
# COMPLETENESS
query = prefixes + """
SELECT (AVG(?propCount) as ?averageProperties)
WHERE {
  {
    SELECT ?surveyPoint (COUNT(DISTINCT ?property) as ?propCount)
    WHERE {
      ?surveyPoint a slo:SurveyPoint .
      ?surveyPoint ?property ?value .
    }
    GROUP BY ?surveyPoint
  }
}

"""
results = kg.query(query)
for row in results:
    print(row)


(rdflib.term.Literal('6.999947324062368310155920775', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')),)


In [25]:
query = prefixes + """
SELECT ?nuts2Region (COUNT(DISTINCT ?surveyPoint) AS ?surveyPointsCount)
WHERE {
  ?surveyPoint a slo:SurveyPoint ;
               slo:surveyLocation ?nuts2Region .
  ?nuts2Region a slo:NUTS2 .
}
GROUP BY ?nuts2Region

"""
results = kg.query(query)
for row in results:
    print(row)


In [24]:
# transform to dataframe only include the NUT code
df = pd.DataFrame(results.bindings)
df.head()


""


In [21]:
query = prefixes + """
PREFIX slo: <https://w3id.org/soilkg/ontology/>
PREFIX ramon: <http://rdfdata.eionet.europa.eu/ramon/ontology/>

SELECT ?nuts2Region (COUNT(DISTINCT ?cropMeasurement) AS ?cropMeasurementsCount)
WHERE {
  ?cropMeasurement a slo:CropMeasurement ;
                   slo:CropMeasurementLocation ?nuts2Region .
  ?nuts2Region a slo:NUTS2 .
}
GROUP BY ?nuts2Region

"""
results = kg.query(query)
for row in results:
    print(row)